For the demo run the cells sequentially. At the end there are a couple of examples showing checkins in the city
of Edinburgh and Las Vegas. You can visualize checkins for different cities. To get the list of different cities refer to the get_location_checkin_data method.

Note that you should have downloaded and unzipped the data ('mapping-yelp-data') as indicated in the README.md

In [1]:
import gmaps
import sys
import json
import pandas as pd
import datetime
sys.path.append('../src')
import yelp_data
import average_rating

In [2]:
def setup_map(api_key_path='../mapping-yelp-data/api-keys/maps-api.key'):
    """
    Sets up the geographical map.
    """
    with open(api_key_path) as f:
        api_key = f.read().strip()
    gmaps.configure(api_key)
    data_map = gmaps.Map()
    return data_map

def get_location_checkin_data(checkin_df, business_data, city='All'):
    """
    :param checkin_df:
    :param business_df:
    :param city: City for which you want to get the location data. By default location data
    for all the cities is returned
    Different cities that you can visualize: ['Edinburgh', 'Karlsruhe', 'Montreal', 'Waterloo', 
                                              'Pittsburgh', 'Urbana-Champaign', 'Phoenix', 
                                              'Las Vegas', 'Madison']
    :return: Location data for the specified city.
    """
    if city != 'All':
        mask = business_data['city'] == city
        business_df = pd.DataFrame(business_data[mask])
    else:
        business_df = pd.DataFrame(business_data)
    checkin_location_data = pd.merge(checkin_df, business_df, on='business_id', how='inner')
    business_locations_df = data.get_business_data_locations(checkin_location_data)
    business_locations_checkins = [(x, y, z) for x, y, z in 
                          zip(business_locations_df['latitude'], 
                              business_locations_df['longitude'],
                              checkin_location_data['count_checkins'])]
    return business_locations_checkins

def get_map(business_locations, point_radius=3):
    """
    :param business_locations: (latitude, longitude) coordinates of businesses
    :return: Geographical map with a heatmap of different checkins
    """
    data_map = setup_map()
    weighted_heatmap_layer = gmaps.WeightedHeatmap(data=business_locations)
    weighted_heatmap_layer.max_intensity = 5
    weighted_heatmap_layer.point_radius = point_radius
    data_map.add_layer(weighted_heatmap_layer)
    return data_map

def checkin_counts(checkin_info):
    """
    :param checkin_info: dictonary containing number of checkins for different blocks
    of time
    :return: sum of all checkins.
    """
    return sum(checkin_info.values())

In [3]:
# Loading data in memory to save time
data = yelp_data.YelpData(datapath='../mapping-yelp-data/yelp_dataset_challenge_academic_dataset/')
checkin_data = data.get_checkin_dataframe()
checkin_data['count_checkins'] = checkin_data['checkin_info'].map(checkin_counts)
business_data = data.get_business_dataframe()

### Heatmap for checkins. The heatmap is weighed according to the number of checkins at a particular place.

In [4]:
# Example showing check-ins in Edinburgh
business_checkin_data = get_location_checkin_data(checkin_data, business_data, city='Edinburgh')
business_map = get_map(business_checkin_data)
# You can zoom in, zoom out with controls in bottom right. You can also switch to a satellite mode.
business_map

In [5]:
# Example showing check-ins in Las Vegas
business_checkin_data = get_location_checkin_data(checkin_data, business_data, city='Las Vegas')
business_map = get_map(business_checkin_data, point_radius=2) # using a smaller point_radius
business_map